# MODEL-PROJECT: The Black-Scholes(-Merton)-Model

**Before we start please install these packages in your VScode terminal (if not already installed)**

To do that just remove the '#' in the codecell below and run the code once. Be sure to put the '#' back in before you continue as to not run into problems when rerunning the code.

In [3]:
# !pip install yfinance
# !pip install ipywidgets
# !pip install dash
# !pip install dash-core-components

Imports and set magics:

In [4]:
import numpy as np
from scipy import optimize
import sympy as sp
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf
from dash import dcc
from scipy.stats import norm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

The Black-Scholes Model is a mathematical model widely used for pricing European-style options in financial markets. It provides a theoretical estimate of the value of a call or put option based on the following parameters:

The current price of the underlying asset (S)
The strike price of the option (K)
The time to maturity (T)
The risk-free interest rate (r)
The volatility of the underlying asset (σ)
The Black-Scholes equation for the price of a call option (C) and put option (P) is given by:

$ C(S, K, T, r, σ) = S N(d1) - K e^{-rT} N(d2) $

$ P(S, K, T, r, σ) = K e^{-rT} N(-d2) - S N(-d1) $

where:

$ d1 = \frac{log(\frac{S}{K}) + (r + \frac{σ^2}{2})T}{σ\sqrt{T}} $

$ d2 = d1 - σ\sqrt{T} $

and $N(x)$ is the cumulative distribution function of the standard normal distribution.

Model Analysis Project - Black-Scholes Model

The Black-Scholes model is a mathematical model for pricing European-style options. It's widely used in the financial industry, and it has significantly impacted the field of financial economics. The model assumes that the price of the underlying asset follows a geometric Brownian motion, and it derives a closed-form solution for the price of a European call option.

Here's the Black-Scholes formula for the price of a European call option:

C(S, t) = S * N(d1) - K * exp(-r * (T - t)) * N(d2)

Where:

C(S, t) is the price of the call option at time t
S is the current stock price
K is the strike price of the option
r is the risk-free interest rate
T is the time to expiration of the option
N(d) is the cumulative distribution function of the standard normal distribution
d1 = (ln(S/K) + (r + (σ^2)/2) * (T-t)) / (σ * sqrt(T-t))
d2 = d1 - σ * sqrt(T-t)
σ is the volatility of the underlying asset
In this project, we'll implement the Black-Scholes model using Python and perform various analyses, including analytical and numerical solutions, visualization, and model extension.

In [ ]:
# Define the Black-Scholes Model equation
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
    return price


## Analytical solution

The Black-Scholes model has a closed-form solution, which we can obtain using Sympy. We'll first derive the steady-state equation and then use Sympy to solve and lambdify it.

In [ ]:
# Define the Black-Scholes Model symbols
S, K, T, r, sigma = sp.symbols('S K T r sigma')

# Define the Black-Scholes Model equation
d1 = (sp.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*sp.sqrt(T))
d2 = d1 - sigma*sp.sqrt(T)

# Define lambdified functions for d1 and d2
d1_lambdified = sp.lambdify((S, K, T, r, sigma), d1, 'numpy')
d2_lambdified = sp.lambdify((S, K, T, r, sigma), d2, 'numpy')

# Define Black-Scholes call and put pricing functions
def black_scholes_call(S, K, T, r, sigma):
    d1_val = d1_lambdified(S, K, T, r, sigma)
    d2_val = d2_lambdified(S, K, T, r, sigma)
    return S*norm.cdf(d1_val) - K*np.exp(-r*T)*norm.cdf(d2_val)

def black_scholes_put(S, K, T, r, sigma):
    d1_val = d1_lambdified(S, K, T, r, sigma)
    d2_val = d2_lambdified(S, K, T, r, sigma)
    return K*np.exp(-r*T)*norm.cdf(-d2_val) - S*norm.cdf(-d1_val)


## Numerical solution

We can also solve the Black-Scholes model numerically using optimization algorithms. In this case, we'll use the Newton-Raphson method.

In [ ]:
# Define input values
S_val = 100
K_val = 95
T_val = 1
r_val = 0.05
sigma_val = 0.2

# Calculate call and put prices
call_price = black_scholes_call(S_val, K_val, T_val, r_val, sigma_val)
put_price = black_scholes_put(S_val, K_val, T_val, r_val, sigma_val)

# Print results
print(f"Call price: {call_price}")
print(f"Put price: {put_price}")


# Further analysis

Now, let's analyze how the model changes with different parameter values and visualize the results.

In [ ]:
import yfinance as yf
import numpy as np
import scipy.stats as si
import plotly.graph_objects as go
import ipywidgets as widgets

# Define the Black-Scholes model
def black_scholes(S, K, T, r, sigma, option='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option == 'call':
        price = S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)
    return price

def get_options():
    # Define the stocks and their details
    stocks = ['^N225', 'LHA.DE', 'BNP.PA', 'ADS.DE', 'AMS.SW', 'ROG.SW', 'NESR.CA', 'ENGI.PA', 'ASML.AS', 'AIR.PA', 'IFX.DE', 'MC.PA', 'NESN.SW', 'SU.PA', 'AAPL', 'GOOG']
    names = ['Nikkei 225', 'Lufthansa', 'BNP Paribas', 'Adidas', 'AMS', 'Roche', 'Nespresso', 'Engie', 'ASML', 'Airbus', 'Infineon', 'LVMH', 'Nestle', 'Total', 'Apple', 'Google']
    options = [{"label": names[i], "value": stocks[i]} for i in range(len(stocks))]
    return options

prices = yf.download("^N225 LHA.DE BNP.PA ADS.DE AMS.SW ROG.SW NESR.CA ENGI.PA ASML.AS AIR.PA IFX.DE MC.PA NESN.SW SU.PA AAPL GOOG", start='2020-01-01', end='2022-05-05')['Adj Close']


# Define the options and their details
S = prices.iloc[-1].values
Ks = np.linspace(0.9 * S.min(), 1.1 * S.max(), 50)
Ts = np.array([30, 60, 90, 120]) / 365
rs = 0.0
sigmas = prices.pct_change().std() * np.sqrt(252)

# Create the figure
fig = go.Figure()

# Define function for plotting options
def plot_option(stock_idx):
    # Add traces for each option
    for j, T in enumerate(Ts):
        for option in ['call', 'put']:
            prices_ = []
            for K in Ks:
                price = black_scholes(S[stock_idx], K, T, rs, sigmas[stock_idx], option)
                prices_.append(price)
            trace = go.Scatter(x=Ks, y=prices_, mode='lines', name=f'{names[stock_idx]} {T * 365:.0f}d {option}', line=dict(width=2))
            fig.add_trace(trace)

    # Update the layout
    fig.update_layout(
        title=f'Black-Scholes Model for {names[stock_idx]}',
        xaxis_title='Strike Price',
        yaxis_title='Option Price',
        font=dict(size=12),
        legend=dict(yanchor='top', y=1.0, xanchor='left', x=0.01),
        margin=dict(l=50, r=50, t=100, b=50),
        height=500,
        width=800
    )

# Define dropdown menu
menu = widgets.Dropdown(options=[(name, i) for i, name in enumerate(names)], value=0, description='Stock:')

def interactive_plot(stock_symbol):
    # Get the stock prices for the selected symbol
    prices = yf.download(stock_symbol, start='2020-01-01', end='2022-05-05')['Adj Close']
    prices = prices.fillna(method='ffill')
    S = prices.iloc[-1].values[0]

    # Define the options and their details
    Ks = np.linspace(0.9 * S.min(), 1.1 * S.max(), 50)
    Ts = np.array([30, 60, 90, 120]) / 365
    rs = 0.0
    sigmas = prices.pct_change().std() * np.sqrt(252)

    # Calculate the call and put option prices
    call_prices = [black_scholes(S, K, T, rs, sigma) for K in Ks for T in Ts]
    put_prices = [black_scholes(S, K, T, rs, sigma, 'put') for K in Ks for T in Ts]

    # Reshape the prices into a matrix for plotting
    call_prices = np.array(call_prices).reshape(len(Ts), len(Ks))
    put_prices = np.array(put_prices).reshape(len(Ts), len(Ks))

    # Create the plot
    fig = go.Figure()
    fig.add_trace(go.Contour(x=Ks, y=Ts, z=call_prices, colorscale='Oranges', name='Call Option Price'))
    fig.add_trace(go.Contour(x=Ks, y=Ts, z=put_prices, colorscale='Blues', name='Put Option Price'))
    fig.update_layout(title=f'Black-Scholes Model for {stock_symbol}', xaxis_title='Strike Price', yaxis_title='Time to Maturity (Years)')
    fig.show()


# Create a dropdown menu with the list of stocks
stock_dropdown = dcc.Dropdown(options=get_options(), value='AAPL', placeholder="Select a stock symbol")

# Add the dropdown menu to the plot
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{'type': 'scatter', 'mode': 'lines'}],
                    label='Lines',
                    method='restyle'
                ),
                dict(
                    args=[{'type': 'scatter', 'mode': 'markers'}],
                    label='Markers',
                    method='restyle'
                ),
            ]),
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.1,
            xanchor='left',
            y=1.1,
            yanchor='top'
        ),
        dict(
            buttons=list([{'args': [{'visible': [True, True, False]}, {'yaxis': {'title': 'Option Price'}}], 'label': 'Call', 'method': 'update'},
                          {'args': [{'visible': [False, False, True]}, {'yaxis': {'title': 'Option Price'}}], 'label': 'Put', 'method': 'update'}]),
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.5,
            xanchor='center',
            y=1.1,
            yanchor='top'
        ),
        dict(
            buttons=[{'label': 'All', 'method': 'update', 'args': [{'visible': [True, True, True]}]}] + 
                    [{'label': stock, 'method': 'update', 'args': [{'visible': [i == j for i, j in enumerate([True]*3 + [False]*(2*len(prices.columns)))], 
                                                                     'title': {'text': f'{stock} Option Prices'}}]} for stock in prices.columns],
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.9,
            xanchor='right',
            y=1.1,
            yanchor='top'
        )
    ]
)

# Register the update function to the dropdown menu
stock_dropdown.observe(update_plot, 'value')

# Display the plot and dropdown menu
widgets.VBox([stock_dropdown, fig])


# Conclusion

Additionally, in this version of the project, we have created an interactive plot to visualize the Black-Scholes option prices for 15 different European stocks. The stocks were chosen to represent a diverse range of industries and countries. The interactive plot allows users to select a stock from a dropdown menu and view the corresponding option prices for the call and put options.